Import libraries

In [1]:
from jetcam.csi_camera import CSICamera
import numpy
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

Create dataset instance

In [2]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

Split dataset into train and test sets

In [3]:
n_samples_test = 20
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - n_samples_test, n_samples_test])

Create data loaders to load data in batches

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

Define the neural network

In [5]:
model = models.alexnet(pretrained=True)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 8)
device = torch.device('cuda')
model = model.to(device)

Train the neural network

In [6]:
NUM_EPOCHS = 15
BEST_MODEL_PATH = 'recognition_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    correct = 0.0
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        #test_error_count += F.cross_entropy(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
    test_accuracy = float(correct) / float(len(test_dataset))
    #test_error = float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    #if test_accuracy > best_accuracy:
torch.save(model.state_dict(), BEST_MODEL_PATH)
       # best_accuracy = test_accuracy

0: 1.000000
1: 1.000000
2: 1.000000
3: 1.000000
4: 1.000000
5: 1.000000
6: 1.000000
7: 1.000000
8: 1.000000
9: 1.000000
10: 1.000000
11: 1.000000
12: 1.000000
13: 1.000000
14: 1.000000
